In [22]:
import sys
import pathlib
pth=str(pathlib.Path().absolute())
sys.path.append(('\\').join(pth.split('\\')[:-1])+"\\Utils")
from Utilities import *
import networkx as nx
import matplotlib.pyplot as plt
import copy

In [9]:
def get_graph_from_image(image,mask,desired_nodes=20):
    SD,segments=get_Statistical_Descriptors_(image,mask,n_segments=desired_nodes)
    nodes=np.array(list(SD))[:]
    node_features=np.vectorize(lambda SD,node:SD[node])(SD,nodes)
    G = nx.Graph()
    for node in nodes[1:]:
        data=np.array(list(node_features[node].items()))[:,1]
        afeatures=np.concatenate((np.concatenate(data[:13]),data[13:]))
        n_features=afeatures.shape[0]
        G.add_node(node-1, features = afeatures)
    
    vs_right = np.vstack([segments[:,:-1].ravel(), segments[:,1:].ravel()])
    vs_below = np.vstack([segments[:-1,:].ravel(), segments[1:,:].ravel()])
    bneighbors = np.unique(np.hstack([vs_right, vs_below]), axis=1)
    bneighbors=np.delete(bneighbors,np.where(bneighbors[1,:]==0),axis=1)
    bneighbors=np.delete(bneighbors,np.where(bneighbors[0,:]==0),axis=1)-1
    
    for i in range(bneighbors.shape[1]):
        if (bneighbors[0,i] != bneighbors[1,i]):
            G.add_edge(bneighbors[0,i],bneighbors[1,i])
    
    for node in nodes[1:]:
        G.add_edge(node-1,node-1)
    
    n = len(G.nodes)
    m = len(G.edges)
    h = np.zeros([n,n_features])
    edges = np.zeros([2*m,2])
    for e,(s,t) in enumerate(G.edges):
        edges[e,0] = s
        edges[e,1] = t
        
        edges[m+e,0] = t
        edges[m+e,1] = s
    for i in G.nodes:
        h[i,:] = G.nodes[i]["features"]
    return SD,G, h, edges

def sample_central(SD,G,num=5,maxdeg=3):
    centers=np.vectorize(pyfunc=lambda i,SD: np.array([SD[i]["x_mean"],SD[i]["y_mean"]]),
             signature="(),()->(j)")(np.arange(1,len(G.nodes)),SD)
    c_node=np.argmin(np.linalg.norm(centers-np.mean(centers,axis=0),axis=1))
    sampled=[c_node]
    deg=1
    th_deg_nei=np.array(list(nx.single_source_shortest_path_length(G, c_node, cutoff=maxdeg).items()))
    #selected=int(centers.shape[0]*samp_frac)
    selected=num
    while len(sampled)!=selected:
        nd=th_deg_nei[np.where(th_deg_nei[:,1]==1)[0]][:,0]
        if (selected-len(sampled))<=len(nd):
            new=np.random.choice(nd,size=selected-len(sampled),replace=False)
        else:
            new=nd
        sampled=sampled+new.tolist()
        deg=deg+1
    return sampled

In [4]:
name='c1anemia-300'
di=('\\').join(pth.split('\\')[:-2])+'\\Data_base\\Imagenes_originales\\'+name+'.jpg'
ROI=('\\').join(pth.split('\\')[:-2])+'\\Data_base\\Sem_Auto\\eye_'+name+'.jpg'
img = io.imread(di)
ROI = io.imread(ROI)
f1=open(('\\').join(pth.split('\\')[:-2])+"\\Data_base\\validcrop.txt","r")
lines=f1.readlines()
linesn=np.array(lines)
linesn=np.delete(lines,np.where(linesn=="\n"))
linesn=linesn.reshape(-1,3)
linesnc=v_replace_err(linesn)
linesnc=np.vectorize(pyfunc=lambda x:np.array([x[0].split('\n')[0]]),signature="(n)->(m)")(linesnc.reshape(-1,1)).reshape(-1,3)
xywh=linesnc[:,:2]
imgnames=linesnc[:,2]
xywh=v_no_spaces(xywh)
xywh=np.vectorize(pyfunc=(lambda x:float(x)))(xywh.reshape(1,-1)[0])
xywh=xywh.reshape(-1,4).astype(int)+1

mr=xywh[np.where(imgnames==(name+".jpg"))][0]
mask=assemble_mask(mr,img,ROI)

In [5]:
SD,G,h,edges=get_graph_from_image(img,mask,desired_nodes=20)

<ipython-input-2-1c7f50201b06>:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  data=np.array(list(node_features[node].items()))[:,1]


In [57]:
centers=np.vectorize(pyfunc=lambda i,SD: np.array([SD[i]["x_mean"],SD[i]["y_mean"]]),
             signature="(),()->(j)")(np.arange(1,len(G.nodes)),SD)
c_node=np.argmin(np.linalg.norm(centers-np.mean(centers,axis=0),axis=1))
sampled=[c_node]
centers.shape[0]
deg=1
th_deg_nei=np.array(list(nx.single_source_shortest_path_length(G, c_node, cutoff=3).items()))
selected=int(centers.shape[0]*0.25)
print(selected)
while len(sampled)!=selected:
    nd=th_deg_nei[np.where(th_deg_nei[:,1]==1)[0]][:,0]
    if (selected-len(sampled))<=len(nd):
        new=np.random.choice(nd,size=selected-len(sampled))
    else:
        new=nd
    print(new)
    sampled=sampled+new.tolist()
    print(sampled)
    deg=deg+1
    




4
[10 19 19]
[12, 10, 19, 19]


[12, 13, 17, 19]

In [11]:
Sampled=sample_central(SD,G,num=5,maxdeg=3)

In [ ]:
lum = np.mean(mask,axis=2).astype(int)
mask1=lum>0
m_slic = slic(img, n_segments=20,sigma=5,mask=mask1)

In [ ]:
plt.plot(centers[:,1],centers[:,0], 'ro')
plt.show()
fig = plt.figure("Superpixels -- %d segments" % (500),figsize=(10,10))
ax = fig.add_subplot(1, 1, 1)
ax.imshow(mark_boundaries(img, m_slic))
ax.plot(centers[:,1],centers[:,0], 'ro')
ax.plot(np.mean(centers[:,1]),np.mean(centers[:,0]), 'bo')
plt.axis("off")

In [ ]:
th_deg_nei=np.array(list(nx.single_source_shortest_path_length(G, c_node, cutoff=3).items()))
len(th_deg_nei[np.where(th_deg_nei[:,1]==1)[0]][:,0])

In [27]:
SDC=copy.deepcopy(SD)
NSD={}

In [28]:
NSD={}
for ID in Sampled:
    NSD[ID]=SD[ID]

In [29]:
NSD

{12: {'rbg_mean': array([0., 0., 0.]),
  'rgb_std': array([19.83668785, 17.48887496,  0.06360356]),
  'rgb_per': array([14.,  2.,  0.]),
  'rgb_mo': array([8.60000000e+01, 7.90000000e+01, 1.45833333e-02]),
  'lab_mean': array([22.05591537, 15.39801769, 12.        ]),
  'lab_std': array([6.57559133, 1.99271511, 0.        ]),
  'lab_per': array([-4.14261919,  2.99262169, 12.        ]),
  'lab_mo': array([16.32126691, 14.0863298 , 12.        ]),
  'hsv_mean': array([ 0.42957686,  0.61558688, 48.25828857]),
  'hsv_std': array([0.07305608, 0.04974373, 6.34180988]),
  'hsv_per': array([ 0.04705882,  0.23921569, 11.57237024]),
  'hsv_mo': array([ 0.37951807,  0.58823529, 44.32331113]),
  'rgb_mean': array([9.96291116e+01, 8.99619140e+01, 2.98979599e-02]),
  'x_mean': 1535.2589155875633,
  'x_std': 35.37613622344871,
  'y_mean': 1489.3891004778063,
  'y_std': 34.29541741590654,
  'N': 14441},
 19: {'rbg_mean': array([0., 0., 0.]),
  'rgb_std': array([22.90521636, 19.00360358,  0.26396224]),
  